In [1]:
# Input the complete matrix, not just lower triangular

coxeter_matrix = matrix([[1,3,2], [3,1,6],[2,6,1]])

dim_v = coxeter_matrix.rank()
# Vector space over the reals
v = VectorSpace(RR,dim_v)

In [2]:
coxeter_matrix

[1 3 2]
[3 1 6]
[2 6 1]

In [3]:
def bilinear(x,y):
    '''Computes the bilinear form of two roots'''
    temp = 0
    for i in range(dim_v):
        for j in range(dim_v):
            temp += x[i]*y[j]*(-cos(pi/coxeter_matrix[i,j])) 
    return temp

def s(i):
    '''Returns the ith simple root'''
    return v.basis()[i]

def simple_reflection(s,x):
    return x-2*bilinear(x,s)*s

def w_action(word_list,x):
    '''Input a word w in the generators as a list and a root \alpha. Returns w(\alpha)'''
    temp_x = x
    word_list.reverse()
    
    for s in word_list:
        temp_x = simple_reflection(s,temp_x)
        
    return temp_x

def inversion_set(w):
    '''Returns the inversion set of an element w as a list'''
    inversionset = [w[0]]

    for i in range(len(w)-1):        
        # note: w[0:i+1] does not include the endpoint i+1
        temp = w[0:i+1]
        inversionset.append(w_action(temp,w[i+1]))
    
    return inversionset

def elementary_roots():
    '''Returns the set of elementary roots'''
    elementary = []
    
    for i in range(dim_v):
        elementary.append(s(i))
    
    temp_n = 1
    counter = 0
    
    while temp_n < len(elementary):
        
        new_root_counter = 0
        
        temp_n = len(elementary)

        for root in elementary[counter:]:
            for i in range(dim_v):

                temp = w_action([s(i)],root)

                if temp[i] > root[i] and temp[i]-root[i] < 2 and temp not in elementary:
                    elementary.append(temp)
                    new_root_counter += 1
        
        counter = len(elementary) - new_root_counter - 1
                      
    return elementary

def elementary_inversion_set(w, elementary):
    '''returns the elementary inversion set of an element, but compute the elementary roots first and input it, 
       to avoid recalculating it everytime.'''
    return inversion_set(w).intersection(elementary)

In [4]:
import math

def dihedral_root_system(r1, r2):
    '''Returns the root system for the dihedral group generated by the reflections r1 and r2
       input is two reflections as a list
    '''
    
    # Get root associated to reflection 1
    middle_of_r1 = int(math.ceil(len(r1)/2))
    
    if middle_of_r1 - 2==0:
        root1 = w_action([r1[0]],r1[middle_of_r1 - 1])
    elif middle_of_r1 == 1:
        root1 = r1[0]
    else:
        root1 = w_action(r1[0:middle_of_r1 - 1],r1[middle_of_r1 - 1])
    
    # Get root associated to reflection 2
    middle_of_r2 = int(math.ceil(len(r2)/2))
    
    if middle_of_r2 - 2==0:
        root2 = w_action([r2[0]],r2[middle_of_r2 - 1])
    elif middle_of_r2 == 1:
        root2 = r2[0]
    else:
        root2 = w_action(r2[0:middle_of_r2 - 1],r2[middle_of_r2 - 1])
    
    roots = [root1, root2]
    reflections = [r1, r2]
    
    temp_n = 1
    counter = 0
    
    while temp_n < len(roots):
        
        new_root_counter = 0
        
        temp_n = len(roots)
        
        if temp_n > 200:
            print("Looks like this will be an infinite dihedral group. I'm going to stop!!")
            break
        
        for root in roots[counter:]:
            for re in reflections:
                new_root = w_action(re,root)
                
                if new_root not in roots:
                    roots.append(new_root)
                    new_root_counter += 1
        
        counter = len(roots) - new_root_counter - 1
        
    return roots

def positive_roots(subgroup):
    
    pos_roots = []
    
    for el in subgroup:
        if sum(el) > 0:
            pos_roots.append(el)
            
    return pos_roots

In [6]:
w_action([s(0),s(2),s(1)], s(2))

(1.00000000000000*sqrt(3), 1.00000000000000*sqrt(3), 2.00000000000000)

In [7]:
w_action([s(2),s(1)],s(2))

(0.000000000000000, 1.00000000000000*sqrt(3), 2.00000000000000)